# Exersice 1 


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
## defining variables

# the URL is the given URL from wich the html code is downloaded
url = "https://www.dw.com/en/why-south-korean-women-arent-having-babies/a-68419317"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")


thinks to find and to extract:
- author
- date
- title
- summary
- main text
- related topics

In [29]:
# the author can be found out with the em tag, since it is the only used case there
soup.em

<em>Edited by: Keith Walker</em>

In [28]:
# the time tag represents an entry with the corresbonding time date
soup.time

<time aria-hidden="true">03/01/2024</time>

In [33]:
# finding out the title
soup.title

"Why South Korean women aren't having babies – DW – 03/01/2024"

In [31]:
# the first paragraph printed on the side is the summary
soup.p

<p class="sc-ivDvhZ iBEcrE sc-bYUneI clArVt sc-kMbQoj bDRlQg sc-cCIIiH kgqfmn sc-BbNMx bBsUeI teaser-text">New statistics show a record low number of children were born last year in South Korea, with women citing a desire for a career and to push back against a male-dominated society as key reasons.</p>

In [29]:
# get the text of the article
soup.find_all("p")

[<p class="cookie__text">We use cookies to improve our service for you. You can find more information in our data protection declaration.</p>,
 <p class="error" style="display:none;">Enter a search term or define search filter.</p>,
 <p>
 The German parliament has voted to approve cannabis for private use, within limits. The opposition and some experts are against it. What is now allowed, what remains prohibited?
 <span></span>
 </p>,
 <p>
 Huge attendances have quickly become commonplace in the WSL, the top division in English women's football. While the league is held up as an example to Germany and others, there may be dangers in such rapid growth.
 <span></span>
 </p>,
 <p>
 Ma Ying-jeou's "journey of peace" aims to calm cross-strait tensions. He is expected to meet Chinese President Xi Jinping, though the meeting has yet to be confirmed.
 <span></span>
 </p>,
 <p>
 President Recep Tayyip Erdogan's conservative Justice and Development Party (AKP) has suffered a significant blow in 

In [32]:
# the tag asside returns other topics related to the article
soup.aside

<aside class="sc-dktgqL jiTpiJ start"><a class="sc-ivDvhZ iBEcrE sc-ivDvhZ iBEcrE sc-iJfdHH djgAGb sc-ezGUZh kTEIUz sc-iAfmwU gMPYoa sc-cUfYYG kOVITf sc-khksUn cjmyLp sc-byiLte bfEvOw" href="/en/minorities/t-50781525" role="button" tabindex="0">Minorities</a><a class="sc-ivDvhZ iBEcrE sc-ivDvhZ iBEcrE sc-iJfdHH djgAGb sc-ezGUZh kTEIUz sc-iAfmwU gMPYoa sc-cUfYYG kOVITf sc-khksUn cjmyLp sc-byiLte bfEvOw" href="/en/womens-rights/t-17455067" role="button" tabindex="0">Women's rights</a><a class="sc-ivDvhZ iBEcrE sc-ivDvhZ iBEcrE sc-iJfdHH djgAGb sc-ezGUZh kTEIUz sc-iAfmwU gMPYoa sc-cUfYYG kOVITf sc-khksUn cjmyLp sc-byiLte bfEvOw" href="/en/south-korea/t-63357867" role="button" tabindex="0">South Korea</a></aside>

In [3]:
# for further use with the function .getText() the html boilerplate can be removed only the pure text gets displayed
print("Output with the .getText Method: " + soup.time.getText())

# the string now has to be converted into a time-datatype to make the future work more easy
# the further analysis of the data is not topic of this exersice and will be conducted in futher research

Output with the .getText Method: 03/01/2024


# Exersice 2

In [24]:
import datetime
import time

In [4]:
url = "https://www.dw.com/search/?languageCode=en"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

In [18]:
latest_date = soup.find_all(name="span", attrs={"class" : "date"})[-1].getText()

# the button that loades the next entries has the class="addPage"



in the following the task will be split into two code-blocks
in the first block all URL's to articles since 01.03.2024 will be collected

in the second code block the relevant data will be extracted from all the links

In [26]:
articleLinks = []
page = 30
more_pages = True

while more_pages == True:

    print(f"Now scraping page {page}")

    url = f"https://www.dw.com/search/?languageCode=en&sort=DATE&resultsCounter={page}"
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    
    news = soup.select("div.searchResult")
    articleLinks.append([article.a["href"] for article in news])


    ## abbruchbedingung wenn das Datum des letzten Artikels vor dem 01.03.2024 liegt
    if datetime.datetime.strptime(latest_date, "%d.%m.%Y") < datetime.datetime.strptime("2024-03-01", "%Y-%m-%d"):
        more_pages = False

    if soup.select("a.addPage"):
        page += 20
    else:
        more_pages = False

    time.sleep(1)   

articleLinks

Now scraping page 30
Now scraping page 50
Now scraping page 70
Now scraping page 90
Now scraping page 110
Now scraping page 130
Now scraping page 150
Now scraping page 170
Now scraping page 190
Now scraping page 210
Now scraping page 230
Now scraping page 250
Now scraping page 270
Now scraping page 290
Now scraping page 310
Now scraping page 330
Now scraping page 350
Now scraping page 370
Now scraping page 390
Now scraping page 410
Now scraping page 430
Now scraping page 450
Now scraping page 470
Now scraping page 490
Now scraping page 510
Now scraping page 530
Now scraping page 550
Now scraping page 570
Now scraping page 590
Now scraping page 610
Now scraping page 630
Now scraping page 650
Now scraping page 670
Now scraping page 690
Now scraping page 710
Now scraping page 730
Now scraping page 750


KeyboardInterrupt: 

In [33]:
i = 0
result = []

for article in articleLinks:
    print(f"Now scraping page {i} with {len(article)} articles")

    url = f"https://www.dw.com{article[i]}"
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    data = {}
    data["author"] = soup.em.getText()
    data["date"] = soup.time.getText()
    data["title"] = soup.title.getText()
    data["summary"] = soup.p.getText()
    data["main_text"] = soup.find_all("p")
    data["related_topics"] = soup.aside.getText()

    result.append(data)

    i += 1


Now scraping page 0 with 30 articles
Now scraping page 1 with 50 articles


AttributeError: 'NoneType' object has no attribute 'getText'

In [34]:
result

[{'author': 'This article was originally written in German.',
  'date': '04/01/2024',
  'title': 'Cannabis is now legal in Germany — within limits – DW – 04/01/2024',
  'summary': 'The German parliament has voted to approve cannabis for private use, within limits. The opposition and some experts are against it. What is now allowed, what remains prohibited?',
  'main_text': [<p class="sc-ivDvhZ iBEcrE sc-bYUneI clArVt sc-kMbQoj bDRlQg sc-cCIIiH kgqfmn sc-BbNMx bBsUeI teaser-text">The German parliament has voted to approve cannabis for private use, within limits. The opposition and some experts are against it. What is now allowed, what remains prohibited?</p>,
   <p>For supporters, <a class="internal-link" href="/en/germanys-bundestag-votes-for-cannabis-legalization/live-68348345">last month's decision in the Bundestag</a> finally marks the end of criminalization, while opponents see an increased risk of young people turning to harder drugs even more than before: With the combined votes 